In [1]:
from data_loader import load_data

greedy  → tabu search → simulated annealing

This solution combines three optimization techniques: Greedy, Simulated Annealing (SA), and Tabu Search. First, a quick initial solution is generated using a greedy approach, which assigns each customer to the nearest depot that can accommodate their demand. Next, Simulated Annealing is applied to iteratively improve the solution by exploring neighboring solutions with a probability of accepting worse solutions, which decreases over time to avoid local minima. Finally, Tabu Search further refines the solution by maintaining a tabu list that prevents revisiting recently explored solutions, encouraging exploration of new, potentially better solutions. Together, these techniques balance exploration and exploitation, leading to an optimized solution for the facility location problem.

In [3]:
import random
import numpy as np

def total_cost(solution, setup_costs, costs):
    depot_cost = sum(setup_costs[d] for d in solution['open_depots'])
    transport_cost = sum(costs[i][solution['assignment'][i]] for i in range(len(solution['assignment'])))
    return depot_cost + transport_cost

def delta_cost(old_solution, new_solution, setup_costs, costs):
    old_open = set(old_solution['open_depots'])
    new_open = set(new_solution['open_depots'])
    depot_diff = sum(setup_costs[d] for d in new_open - old_open) - sum(setup_costs[d] for d in old_open - new_open)
    assignment_old = old_solution['assignment']
    assignment_new = new_solution['assignment']
    transport_diff = sum(
        costs[i][assignment_new[i]] - costs[i][assignment_old[i]]
        for i in range(len(assignment_old)) if assignment_new[i] != assignment_old[i]
    )
    return depot_diff + transport_diff

def is_feasible(assignment, depot_capacities, customer_demands, n_depots):
    depot_loads = [0] * n_depots
    for i, d in enumerate(assignment):
        depot_loads[d] += customer_demands[i]
    return all(depot_loads[i] <= depot_capacities[i] for i in range(n_depots))

def initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs):
    assignment = [-1] * n_customers
    remaining_capacity = depot_capacities[:]
    for i in range(n_customers):
        sorted_depots = sorted(range(n_depots), key=lambda d: costs[i][d])
        for d in sorted_depots:
            if remaining_capacity[d] >= customer_demands[i]:
                assignment[i] = d
                remaining_capacity[d] -= customer_demands[i]
                break
    open_depots = list(set(assignment))
    return {'assignment': assignment, 'open_depots': open_depots}

def get_neighbors_limited(solution, n_depots, depot_capacities, customer_demands, k=3):
    neighbors = []
    assignment = solution['assignment']
    for _ in range(k):
        i = random.randint(0, len(assignment)-1)
        current_d = assignment[i]
        for d in range(n_depots):
            if d != current_d:
                new_assignment = assignment[:]
                new_assignment[i] = d
                if is_feasible(new_assignment, depot_capacities, customer_demands, n_depots):
                    open_depots = list(set(new_assignment))
                    neighbors.append({'assignment': new_assignment, 'open_depots': open_depots})
    return neighbors

def simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                        initial_temp=1000, cooling_rate=0.95, max_iter=100, neighbor_k=5):
    greedy = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
    current = tabu_search_optimized(greedy, n_depots, depot_capacities, customer_demands, setup_costs, costs)   
    best = current
    temp = initial_temp
    for _ in range(max_iter):
        neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
        if not neighbors:
            break
        candidate = random.choice(neighbors)
        cost_diff = total_cost(candidate, setup_costs, costs) - total_cost(current, setup_costs, costs)
        if cost_diff < 0 or random.random() < np.exp(-cost_diff / temp):
            current = candidate
            if total_cost(current, setup_costs, costs) < total_cost(best, setup_costs, costs):
                best = current
        temp *= cooling_rate
    best_cost = total_cost(best, setup_costs, costs)
    return best, best_cost


def tabu_search_optimized(initial_sol, n_depots, depot_capacities, customer_demands,
                          setup_costs, costs, max_iter=50, tabu_tenure=5, neighbor_k=10, runs=3):
    best_global = None
    for _ in range(runs):
        current = initial_sol
        best = current
        best_cost = total_cost(best, setup_costs, costs)
        tabu_list = []
        for _ in range(max_iter):
            neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
            neighbors = [n for n in neighbors if n['assignment'] not in tabu_list]
            if not neighbors:
                break
            candidate = min(neighbors, key=lambda s: delta_cost(current, s, setup_costs, costs))
            cost_candidate = total_cost(candidate, setup_costs, costs)
            if cost_candidate < best_cost:
                best = candidate
                best_cost = cost_candidate
            tabu_list.append(candidate['assignment'])
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            current = candidate
        if best_global is None or total_cost(best, setup_costs, costs) < total_cost(best_global, setup_costs, costs):
            best_global = best
    return best_global


In [4]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  801525.9
Assignment :  {'assignment': [7, 6, 0, 24, 7, 0, 1, 5, 7, 7, 3, 10, 5, 0, 6, 7, 19, 8, 20, 6, 20, 6, 10, 0, 11, 10, 12, 10, 19, 0, 15, 10, 15, 16, 11, 11, 17, 5, 23, 24, 19, 20, 7, 6, 22, 23, 23, 11, 24, 11], 'open_depots': [0, 1, 3, 5, 6, 7, 8, 10, 11, 12, 15, 16, 17, 19, 20, 22, 23, 24]}


In [5]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  920309.8625
Assignment :  {'assignment': [34, 34, 25, 48, 15, 5, 36, 12, 46, 15, 10, 22, 12, 25, 34, 15, 16, 22, 41, 19, 16, 21, 22, 5, 24, 25, 26, 22, 22, 5, 30, 22, 30, 33, 34, 16, 36, 12, 46, 48, 22, 41, 15, 21, 44, 46, 46, 19, 48, 24], 'open_depots': [5, 10, 12, 15, 16, 19, 21, 22, 24, 25, 26, 30, 33, 34, 36, 41, 44, 46, 48]}


In [6]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  29618.163
Assignment :  {'assignment': [197, 95, 84, 125, 24, 53, 198, 93, 34, 198, 178, 198, 111, 101, 162, 47, 66, 23, 148, 2, 158, 137, 71, 126, 32, 48, 163, 95, 106, 34, 184, 178, 48, 93, 52, 102, 150, 20, 40, 3, 79, 149, 194, 147, 158, 84, 19, 126, 133, 126, 165, 125, 158, 32, 162, 129, 13, 79, 126, 53, 154, 162, 102, 53, 78, 171, 165, 121, 70, 38, 166, 49, 87, 154, 147, 25, 113, 5, 20, 81, 198, 138, 111, 178, 53, 195, 150, 24, 150, 2, 45, 24, 75, 184, 128, 102, 169, 176, 12, 121, 124, 165, 38, 168, 53, 25, 8, 13, 198, 23, 93, 49, 47, 95, 19, 53, 113, 163, 166, 66, 196, 52, 23, 0, 24, 126, 124, 28, 34, 107, 13, 84, 153, 5, 169, 163, 75, 31, 105, 40, 168, 106, 21, 176, 149, 45, 0, 110, 71, 70, 12, 113, 31, 0, 105, 154, 153, 12, 196, 3, 32, 71, 153, 86, 133, 86, 12, 198, 110, 195, 66, 109, 144, 70, 86, 125, 149, 176, 176, 21, 28, 66, 157, 3, 149, 101, 139, 198, 27, 79, 78, 49, 144, 19, 139, 107, 40, 137, 22, 13], 'open_depots': [0, 2, 3, 5, 8, 12, 13, 19, 20, 21, 22, 2

In [7]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  73085.42680999999
Assignment :  {'assignment': [134, 53, 292, 256, 125, 90, 225, 275, 35, 100, 221, 275, 240, 235, 186, 49, 161, 4, 252, 118, 238, 268, 42, 226, 103, 169, 28, 147, 268, 126, 220, 51, 236, 36, 237, 48, 19, 132, 284, 107, 22, 131, 289, 12, 219, 256, 284, 48, 83, 11, 74, 71, 273, 77, 187, 175, 148, 264, 59, 290, 132, 122, 28, 28, 181, 208, 92, 126, 178, 219, 140, 253, 62, 204, 172, 148, 42, 235, 234, 58, 125, 297, 22, 219, 12, 252, 268, 140, 96, 12, 13, 27, 217, 101, 296, 28, 258, 69, 84, 196, 141, 100, 166, 219, 181, 164, 69, 272, 282, 19, 266, 132, 141, 277, 129, 201, 82, 18, 211, 84, 93, 163, 204, 175, 240, 296, 296, 62, 219, 103, 277, 264, 221, 113, 297, 276, 292, 187, 281, 2, 27, 278, 42, 20, 187, 202, 19, 122, 134, 117, 46, 35, 70, 209, 93, 186, 13, 206, 103, 257, 210, 172, 135, 19, 119, 11, 27, 232, 118, 289, 253, 103, 54, 12, 268, 284, 164, 231, 140, 155, 0, 204, 219, 264, 288, 90, 188, 275, 2, 113, 52, 238, 268, 28, 210, 46, 54, 288, 22, 0, 172, 96, 

In [8]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  80761.02428
Assignment :  {'assignment': [432, 79, 473, 419, 269, 127, 87, 51, 60, 324, 260, 38, 31, 66, 176, 344, 161, 168, 471, 276, 94, 458, 499, 154, 174, 290, 76, 199, 437, 458, 144, 295, 180, 372, 64, 442, 141, 320, 269, 425, 427, 243, 9, 286, 369, 31, 149, 471, 245, 484, 35, 140, 256, 467, 166, 338, 384, 385, 276, 94, 104, 121, 446, 221, 379, 285, 351, 381, 133, 285, 460, 472, 184, 304, 115, 487, 187, 158, 351, 429, 94, 243, 54, 135, 12, 121, 350, 337, 470, 128, 320, 445, 272, 234, 176, 176, 403, 111, 2, 163, 268, 84, 144, 426, 109, 497, 171, 240, 493, 183, 262, 473, 8, 280, 496, 123, 356, 407, 63, 15, 443, 408, 391, 58, 214, 144, 121, 123, 337, 102, 265, 435, 386, 423, 499, 279, 295, 371, 12, 118, 336, 481, 8, 416, 64, 484, 200, 384, 433, 151, 367, 113, 325, 216, 498, 367, 111, 109, 371, 324, 354, 360, 131, 285, 135, 19, 2, 275, 367, 0, 69, 141, 407, 165, 60, 497, 474, 103, 130, 106, 443, 174, 421, 365, 2, 102, 27, 51, 489, 350, 435, 265, 333, 13, 386, 337, 431, 2